# Practica 1

Este notebook esta dedicado a resolver la práctica número 1 del diplomado Ciencia de Datos de la FES Acatlán. 

## Paqueterías

In [1]:
import numpy as np
import pandas as pd
import ast

import unicodedata
import re

## Datos

In [2]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,zip Code,genero
0,0.764,['Shel Silverstein'],0.669,210387,0.1720,NaN,1r3N7MoEGPHJyCjBlWCsx3,0.000000,2.0,0.1280,-13.075,1,The Unicorn,12.0,1962-10-13 00:00:00,0.3270,97.079,0.333,38ee029,NaN
1,0.917,['Mose Allison'],0.693,149547,0.2410,NaN,2RL8EkOOu0QhAikhUUvIRi,0.000014,NaN,0.0824,-14.605,0,If You Live,19.0,1963-04-14 00:00:00,0.0350,120.890,0.715,9p2304,NaN
2,0.926,['水柳仙'],0.183,207173,0.0854,0.0,3kKKEghp2EMh8992pWFylr,0.704000,7.0,0.1120,-26.598,1,鴛鴦夢,23.0,1963-11-14 00:00:00,0.0392,101.482,0.131,2166ee6,NaN
3,0.976,['Dean Martin'],0.472,182653,0.0314,NaN,0j3ideT0tRuIyRhOOUQjYa,0.000015,7.0,0.1500,-18.006,1,Fools Rush In,21.0,1964-06-07 00:00:00,0.0364,135.555,0.194,67t06,NaN
4,0.892,['Doris Day'],0.243,171400,0.3360,NaN,0od9zbD6Bcc8b7dCMRFWBZ,0.000049,2.0,0.1310,-9.812,0,The Christmas Waltz,22.0,1964-08-07 00:00:00,0.0282,89.158,0.238,804p53,NaN


## Parte 1

### A) Realiza el etiquetado de las variables de acuerdo a su tipo de variable

In [3]:
variables_continuas = [v for v in df.dtypes.index if df.dtypes[v] != 'object']
variables_categoricas = ['zip Code','genero']
variables_tipo_fecha = ['release_date']
variables_de_texto = ['name','artists']

In [4]:
variables_continuas_new = ['c_' + x for x in variables_continuas]
variables_categoricas_new = ['v_' + x for x in variables_categoricas]
variables_tipo_fecha_new = ['d_' + x for x in variables_tipo_fecha]
variables_de_texto_new = ['t_' + x for x in variables_de_texto]

In [5]:
df.rename(columns=dict(zip(variables_continuas,variables_continuas_new)),inplace=True)
df.rename(columns=dict(zip(variables_categoricas,variables_categoricas_new)),inplace=True)
df.rename(columns=dict(zip(variables_tipo_fecha,variables_tipo_fecha_new)),inplace=True)
df.rename(columns=dict(zip(variables_de_texto,variables_de_texto_new)),inplace=True)

### B) Eliminar duplicados

In [6]:
df.duplicated().sum()

364

In [7]:
df.drop_duplicates(inplace=True)

### C) Completitud de las variables

In [8]:
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

In [9]:
completitud(df)

,columna,total,completitud
0,c_explicit,62905,62.993794
1,c_key,35759,78.963438
2,c_popularity,3501,97.940407
3,v_genero,1698,99.001088
4,c_valence,0,100.000000
5,c_tempo,0,100.000000
6,c_speechiness,0,100.000000
7,d_release_date,0,100.000000
8,t_name,0,100.000000
9,c_mode,0,100.000000


### D) Elimina las variables que tengan el 20 % o más de valores ausentes

In [10]:
df.drop(columns=['c_explicit', 'c_key'], inplace=True)

### E) ¿Cuántos de los registros en la variable "zip Code" son valores no válidos? , es decir, contienen letras.

In [11]:
sum(df['v_zip Code'].apply(lambda x: 1 if type(x)==str else 0))

38913

### F) Elimina los registros que no tengas un ”zip Code” válido, es decir contenga letras en los valores

In [12]:
aux = df['v_zip Code'][df['v_zip Code'].apply(lambda x: True if type(x)==str else False)].index.tolist()

In [13]:
df.drop(index=aux, inplace=True)

### G) ¿Cuántos de los registros en la variable "genero" son valores no validos? , es decir, contienen letras

In [14]:
sum(df.v_genero.apply(lambda x: 1 if type(x)!=float else 0))

0

In [15]:
df.v_genero.value_counts()

47.0    26374
20.0    24148
50.0    19545
12.0    17772
16.0    17513
21.0    17259
30.0     2571
49.0     2557
48.0     2480
3.0       434
2.0       276
1.0       143
Name: v_genero, dtype: int64

### H) Elimina los registros que no tengas un "genero" válido, es decir contenga letras en los valores

In [16]:
#No hay valores con esas caracteristicas y si los hay se eliminaron en incisos anterionres

### I) Haz una limpieza de la variable "name" , elimina caracteres especiales y todo debe estar en minúsculas

In [17]:
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text

In [18]:
df.t_name = df.t_name.apply(lambda x: clean_text(x))

### J) De la variable "artist" selecciona solo al primer artista que aparezca en la lista además de eliminar caracteres especiales

In [19]:
df.t_artists = df.t_artists.apply(lambda x: clean_text(ast.literal_eval(x)[0]))

### K) Normaliza la variable "genero" de tal forma que obtengas solo 8 categorías

In [20]:
df.v_genero.value_counts()

47.0    26374
20.0    24148
50.0    19545
12.0    17772
16.0    17513
21.0    17259
30.0     2571
49.0     2557
48.0     2480
3.0       434
2.0       276
1.0       143
Name: v_genero, dtype: int64

In [21]:
aux = dict(zip(df["v_genero"].value_counts(1).index[-5:].tolist(),["otro"]*5))
df["v_genero"].replace(aux).value_counts()

47.0    26374
20.0    24148
50.0    19545
12.0    17772
16.0    17513
21.0    17259
otro     5890
30.0     2571
Name: v_genero, dtype: int64

In [22]:
len(df["v_genero"].replace(aux).value_counts())

8

### L) Añade las siguientes columnas a tu tabla base: zip, lat, lng, city, state name utilizando la tabla zips practica

In [23]:

body = client_8ab9e345e41b4561ab7096742fcbbe1b.get_object(Bucket='diplomadocienciadedatos-donotdelete-pr-ni0xvyxds7qosj',Key='zips_practica.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_zips = pd.read_csv(body)


In [24]:
df.rename(columns={'v_zip Code': 'zip'}, inplace=True)

In [25]:
df = df.merge(df_zips[['zip', 'lat', 'lng', 'city', "state_name"]], on='zip', how='left')

In [26]:
df.rename(columns={'zip':'v_zip', 'lat':'c_lat', 'lng':'c_lng', 'city':'v_city', 'state_name':'v_state_name'}, inplace=True)

### M) Convierte las variables "lat" y "lng" en tipo flotante y valida que la información sea consistente.

In [27]:
df[['c_lat','c_lng']].dtypes

c_lat    float64
c_lng    float64
dtype: object

### N) De la variable "city" y "state" elimina los dígitos que se encuentran dentro de las cadenas de texto

In [28]:
df.v_city = df.v_city.apply(lambda x: re.sub(r'[0-9]', '', x))

In [29]:
df.v_state_name = df.v_state_name.apply(lambda x: re.sub(r'[0-9]', '', x))

### O) Crea una nueva variable llamada "state" que este conformada por "city”,”state name"

In [30]:
df['v_state'] = df.v_city + ', ' +  df.v_state_name

### P) Los valores de la nueva variable "state" modificalos de cierta forma que todas sean minúsculas y sin acentos

In [31]:
df.v_state = df.v_state.apply(lambda x: clean_text(x))

### Q) Convierte los valores en la variable "release date" a tipo datetime , además haz el conteo de las que no tienen la estructura necesaria para ser convertida en datetime y elimina esos registros

In [32]:
df.d_release_date = pd.to_datetime(df.d_release_date)

## Parte 2

### A) Obten un DataFrame donde se muestre el conteo de registros por género

In [33]:
df[['id', 'v_genero']].groupby('v_genero').count().rename(columns={'id': 'conteo'})

,conteo
v_genero,
1.0,143
2.0,276
3.0,434
12.0,17772
16.0,17513
20.0,24148
21.0,17259
30.0,2571
47.0,26374


### B) Genera una nueva variable que se llame "duration minutos" que sea el valor de la variable "duration ms" en minutos

In [34]:
df["c_duration_minutos"] = df['c_duration_ms']/60000

### C) ¿Cuáles son las 10 canciones más populares?

In [35]:
df[['c_popularity', 't_name']].sort_values('c_popularity', ascending=False).head(10)

,c_popularity,t_name
53679,100.0,blinding lights
53677,99.0,rockstar feat roddy ricch
53686,97.0,death bed coffee for your head feat beabadoobee
53678,96.0,the scotts
53582,95.0,supalonely
53590,95.0,ily i love you baby feat emilee
53578,95.0,the box
53683,95.0,toosie slide
53689,94.0,rain on me with ariana grande
53688,94.0,gooba


In [36]:
df.t_name[10:20]

10                                             forecast
11                                              granada
12                                      address unknown
13    concerto in the italian style in f major bwv 9...
14                                        mambo retozon
15                                                     
16                    marmaduke short take 3 new take 4
17                                   everybody s rockin
18          all this and heaven 2015 remastered version
19                                        getting drunk
Name: t_name, dtype: object

### D) ¿Cúal es el promedio de duración en minutos y milisegundos?

In [37]:
df[['c_duration_minutos', 'c_duration_ms']].mean()

c_duration_minutos         3.840535
c_duration_ms         230432.114380
dtype: float64

### E) ¿Cuál es el promedio y conteo de "energy" por "genero"?

In [38]:
df[['c_energy', 'v_genero']].groupby('v_genero').agg({'c_energy':['mean','count']})

c_energy       
              mean  count
v_genero                 
1.0       0.496108    143
2.0       0.463160    276
3.0       0.475278    434
12.0      0.487540  17772
16.0      0.481487  17513
20.0      0.484086  24148
21.0      0.483960  17259
30.0      0.483541   2571
47.0      0.486945  26374
48.0      0.487428   2480
49.0      0.482562   2557
50.0      0.487854  19545

### F) ¿Qué canción tiene el "loudness" más bajo y cúal es la canción que tiene el "loud-ness" más alto?

In [39]:
print('cannción con loudness más alto: ', df[['c_loudness','t_name']].sort_values('c_loudness',ascending=False).head(1)['t_name'].tolist())

cannción con loudness más alto:  ['sold']


In [40]:
print('cannción con loudness más bajo: ', df[['c_loudness','t_name']].sort_values('c_loudness').head(1)['t_name'].tolist())

cannción con loudness más bajo:  ['staggerlee has his day at the beach']


### G) Obtenga los 10 percentiles de las variables continuas

In [41]:
df[df.filter(like="c_").columns.tolist()].quantile([.1, .2,.3,.4,.5,.6,.7,.8,.9])

,c_acousticness,c_danceability,c_duration_ms,c_energy,c_instrumentalness,c_liveness,c_loudness,c_mode,c_popularity,c_speechiness,c_tempo,c_valence,c_lat,c_lng,c_duration_minutos
0.1,0.009561,0.301,137600.0,0.130,0.000000,0.0728,-18.9270,0.0,0.0,0.0301,79.5131,0.163,32.092020,-115.599290,2.293333
0.2,0.055900,0.386,162093.0,0.219,0.000000,0.0912,-15.6518,0.0,5.0,0.0334,89.1440,0.275,34.341250,-99.675240,2.701550
0.3,0.153000,0.449,178333.0,0.301,0.000002,0.1050,-13.5560,1.0,19.0,0.0366,97.3370,0.371,36.438142,-95.610210,2.972217
0.4,0.305000,0.502,192406.4,0.387,0.000021,0.1160,-11.9710,1.0,27.0,0.0403,106.0004,0.465,38.167360,-91.919890,3.206773
0.5,0.504000,0.550,207434.5,0.476,0.000198,0.1350,-10.5340,1.0,34.0,0.0450,114.7495,0.548,39.509060,-88.212620,3.457242
0.6,0.696000,0.596,226227.0,0.564,0.002040,0.1670,-9.1870,1.0,40.0,0.0521,122.5690,0.630,40.606020,-84.587938,3.770450
0.7,0.837000,0.643,248596.1,0.659,0.024000,0.2220,-7.8420,1.0,46.0,0.0646,130.5304,0.711,41.574410,-81.656810,4.143268
0.8,0.936000,0.696,278786.2,0.754,0.289000,0.3040,-6.4980,1.0,52.0,0.0924,140.9730,0.795,42.762572,-78.307000,4.646437
0.9,0.984000,0.760,334185.6,0.864,0.832000,0.4060,-5.0450,1.0,61.0,0.1900,161.8349,0.888,44.722860,-74.599710,5.569760


# FIN